<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
# TODO - Words, words, mere words, no matter from the heart.

In [2]:
from __future__ import print_function

maxlen = 80
max_features = 20000
batch_size = 32


from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.optimizers import RMSprop

In [3]:
url = 'https://www.gutenberg.org/files/100/100-0.txt'

In [4]:
!wget 'https://www.gutenberg.org/files/100/100-0.txt'

--2019-12-17 01:28:11--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777367 (5.5M) [text/plain]
Saving to: ‘100-0.txt’

100-0.txt           100%[===================>]   5.51M  9.05MB/s    in 0.6s    

2019-12-17 01:28:14 (9.05 MB/s) - ‘100-0.txt’ saved [5777367/5777367]



In [5]:
import requests

data = requests.get(url)
data.encoding = 'utf-8'

In [6]:
text = data.text

In [7]:

# encode the data as characters
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
maxlen=32
step=5

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('sequences:', len(sentences))

sequences: 1148005


In [9]:
next_chars[1]

'o'

In [10]:
import numpy as np
# Specify x & y

#encoding the sequence data into numpy vectors

#it is faster to update something than create it, so initializing with a 0's array set
#creating 0 numpy arrays, len(sentances) length, and each array will have maxlen number of entries and the length of characters set.

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


#we are looping through all the sentance, preserving the id number i
for i, sentence in enumerate(sentences):
    #we are going to update each of those arrays, and preserve the location integerger of the character in that sequence t
    for t, char in enumerate(sentence):
        #we are appending the sentance id, the character location id and the actual character number and set that equal to 1
        x[i, t, char_indices[char]] = 1
        #taking the sentance id and passing the next character and setting it equal to 1
    y[i, char_indices[next_chars[i]]] = 1

In [11]:
print(x.shape)
print(y.shape)

(1148005, 32, 108)
(1148005, 108)


In [12]:
# Build the LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
# sample function from lecture

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys
import os

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [15]:
model.fit(x, y,
          batch_size=128,
         epochs=5,
         callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
1147904/1148005 [============================>.] - ETA: 0s - loss: 1.7293
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "elf;
At whose conception, till "
elf;
At whose conception, till the stand the sunce and this shall see thee and such the bear me the party the contention.
                                                                                                                                                                                                                                                                                                                    
----- diversity: 0.5
----- Generating with seed: "elf;
At whose conception, till "
elf;
At whose conception, till be made a bear of your drowers with with on her honour of your heart for thee.
                                                                              

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in log


 o .w, hemne wKlhathlende e
tol phathlor mooe ,nhl.
 ,fon.U k N  Aor
 , me pthemy molhpes, pr an   Sn,nOnen

sarevt Ak myoulal.nce   PfAneE  A l ,el,’'le.s
 w.ol h' Aory DahedandereHe netIw mo mer Can
l: delI,El.
sav, IDds, wyNof otceId   an ’y?a N,noMlney  hel,k lhavte Uefn,Lee dH', Sondeereaeyor te  lyge ahewshere r l ntet  y toenhloulaet teerenrem D, hs m.on
----- diversity: 0.5
----- Generating with seed: ", you indicate that you have rea"
, you indicate that you have reange  y mharo,gande U th ynounonLdhuErende cevhuto asrtnhor  thlh  ffrelv crtt  ;ue
hkeefeR,mhoemieknw,ay n re of f nwrIhly ,heli
r
 w 
e tta
lees,nenther no  gEAl g.thuth,thr  Ea dre nokyihersteree thleMuo dolleey tehndeeRethleye cathe enc werewEOngehba thr nhac wor oi Eoure ewthoeyoManhuy Moon,
nrelder ngye Oatheree ho  moman Uv
sw'y e  nhlheesEee n, thuden,LeRhe  ppr asnethhs
----- diversity: 1.0
----- Generating with seed: ", you indicate that you have rea"
, you indicate that you have reatpE oinatre.
omea noned

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN